In [1]:
import numpy as np
import pandas as pd
import torch

from transformers import BertForSequenceClassification, BertTokenizer, AdamW
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

In [ ]:
# Load Data
class TextDataset(Dataset):
    def __init__(self, tokenizer, filepath):
        self.data = pd.read_csv(filepath)
        self.tokenizer = tokenizer
        self.texts = self.data['text'].tolist()  # Update 'text_column_name' with the actual column name for texts
        self.labels = torch.tensor(self.data['label'].tolist())  # Update 'label_column_name' with the actual column name for labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        inputs = self.tokenizer(text, return_tensors='pt', max_length=512, truncation=True, padding='max_length')
        input_ids = inputs['input_ids'].squeeze(0)  # Remove batch dimension
        return input_ids, label
    
# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load data
dataset = TextDataset(tokenizer, 'path_to_your_file.csv')
loader = DataLoader(dataset, batch_size=16, shuffle=True)

## Train Adversarial Model

In [ ]:
def adversarial_training(model, inputs, labels, epsilon=0.01):
    outputs = model(**inputs)
    loss = torch.nn.functional.cross_entropy(outputs.logits, labels)
    model.zero_grad()
    loss.backward()  # Compute gradients

    # Add adversarial noise
    inputs_grad = {key: val.grad.sign() for key, val in inputs.items() if val.requires_grad}
    perturbed_inputs = {key: val + epsilon * inputs_grad[key] for key, val in inputs.items() if val.requires_grad}

    # Re-run the model on perturbed data
    with torch.no_grad():
        perturbed_outputs = model(**perturbed_inputs)
        perturbed_loss = torch.nn.functional.cross_entropy(perturbed_outputs.logits, labels)

    return loss.item(), perturbed_loss.item()

In [ ]:
# Initialize model and tokenizer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
optimizer = AdamW(model.parameters(), lr=5e-5)

# Set model to training mode
model.train()

for epoch in range(4):  # Loop over the dataset multiple times
    total_loss = 0
    total_adv_loss = 0
    for batch_inputs, batch_labels in loader:
        inputs = {'input_ids': batch_inputs, 'labels': batch_labels}
        loss, adv_loss = adversarial_training(model, inputs, batch_labels, epsilon=0.01)
        optimizer.step()
        total_loss += loss
        total_adv_loss += adv_loss

    avg_loss = total_loss / len(loader)
    avg_adv_loss = total_adv_loss / len(loader)
    print(f'Epoch {epoch+1}, Loss: {avg_loss}, Adversarial Loss: {avg_adv_loss}')


In [ ]:
def evaluate_model(model, dataloader):
    model.eval()
    true_labels, predictions = [], []

    with torch.no_grad():
        for batch_inputs, batch_labels in dataloader:
            inputs = {'input_ids': batch_inputs}
            outputs = model(**inputs)
            logits = outputs.logits
            predicted_labels = torch.argmax(logits, dim=1)
            predictions.extend(predicted_labels.numpy())
            true_labels.extend(batch_labels.numpy())

    accuracy = accuracy_score(true_labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='binary')
    conf_matrix = confusion_matrix(true_labels, predictions)

    return accuracy, precision, recall, f1, conf_matrix


accuracy, precision, recall, f1, conf_matrix = evaluate_model(model, loader)
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print('Confusion Matrix:')
print(conf_matrix)